In [9]:
from tensorflow.keras.datasets import mnist
from google.colab.patches import cv2_imshow
import numpy as np
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import argparse
import imutils
import cv2

In [10]:
# from sklearn.metrics import mean_squared_error
!pip install scikit-image
from skimage import measure

In [11]:
def load_az_dataset(datasetPath):
	data = []
	labels = []

	file = open(datasetPath)
	for row in file:

		row = row.split(",")
		label = int(row[0])
		image = np.array([int(x) for x in row[1:]], dtype="uint8")

		image = image.reshape((28, 28))
		data.append(image)
		labels.append(label)
	data = np.array(data, dtype="float32")
	labels = np.array(labels, dtype="int")
	return (data, labels)


In [12]:
def load_mnist_dataset():

	((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
	data = np.vstack([trainData, testData])
	labels = np.hstack([trainLabels, testLabels])
	return (data, labels)

In [13]:
import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2

In [14]:
# Setting up hyperparameters
EPOCHS = 10
INIT_LR = 1e-1
BS = 128

In [15]:
# load the A-Z and MNIST datasets, respectively
print("[INFO] loading datasets...")
(azData, azLabels) = load_az_dataset('/content/drive/MyDrive/CV/A_Z Handwritten Data/A_Z Handwritten Data.csv')
(digitsData, digitsLabels) = load_mnist_dataset()

[INFO] loading datasets...
11493376/11490434 [==============================] - 0s 0us/step


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
azLabels += 10
data = np.vstack([azData, digitsData])
labels = np.hstack([azLabels, digitsLabels])
# Resizing the image to 32 x 32
data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")
# Normalising the data
data = np.expand_dims(data, axis=-1)
data /= 255.0

In [18]:
le = LabelBinarizer()
labels = le.fit_transform(labels)
counts = labels.sum(axis=0)
classTotals = labels.sum(axis=0)
classWeight = {}

for i in range(0, len(classTotals)):
	classWeight[i] = classTotals.max() / classTotals[i]

(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.20, stratify=labels, random_state=42)

In [19]:
aug = ImageDataGenerator(
	rotation_range=10,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	fill_mode="nearest")

In [20]:
# Creating CNN model 
from keras.models import Sequential
#creating model object
model=Sequential()

from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

#adding layers and forming the model
model.add(Conv2D(32,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(32,32,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(64,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))

model.add(Flatten())

model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(36,activation="softmax"))

#compiling
model.compile(loss="categorical_crossentropy", optimizer='adamax',metrics=["accuracy"])


In [ ]:
# training the network
print("[INFO] training network...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS,
	class_weight=classWeight,
	verbose=1)

[INFO] training network...
Epoch 1/10
 141/2765 [>.............................] - ETA: 1:35 - loss: 11.6084 - accuracy: 0.3303

In [ ]:
# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))

In [ ]:
# Low Level Vision
image = cv2.imread('/content/drive/MyDrive/CV/abcd.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# perform edge detection, find contours in the edge map, and sort the
# resulting contours from left-to-right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]
# initialize the list of contour bounding boxes and associated
# characters that we'll be OCR'ing
chars = []

In [ ]:
for c in cnts:
	(x, y, w, h) = cv2.boundingRect(c)
	if (w >= 5 and w <= 150) and (h >= 15 and h <= 120):
		roi = gray[y:y + h, x:x + w]
		thresh = cv2.threshold(roi, 0, 255,
			cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
		(tH, tW) = thresh.shape
		if tW > tH:
			thresh = imutils.resize(thresh, width=32)
		else:
			thresh = imutils.resize(thresh, height=32)
		(tH, tW) = thresh.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)
		# pad the image and force 32x32 dimensions
		padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
			left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
			value=(0, 0, 0))
		padded = cv2.resize(padded, (32, 32))
		# prepare the padded image for classification via our
		# handwriting OCR model
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)
		# update our list of characters that will be OCR'd
		chars.append((padded, (x, y, w, h)))


In [ ]:
# extract the bounding box locations and padded characters
boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")
# OCR the characters using our handwriting recognition model
preds = model.predict(chars)
# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [ ]:
save=[]
bosx=[]
for (pred, (x, y, w, h)) in zip(preds, boxes):	
	# find the index of the label with the largest corresponding
	# probability, then extract the probability and label
	i = np.argmax(pred)
	prob = pred[i]
	label = labelNames[i]
	# draw the prediction on the image
	print("[INFO] {} - {:.2f}%".format(label, prob * 100))
	print(prob)
	print(label)
	if(prob>0.99):
	  save.append(label)
	  bosx.append((x,y,w,h))

In [ ]:
# General information
print(len(save))
print(len(bosx))
print(save)
print(bosx[17])

In [ ]:
image1 = cv2.imread('/content/Hello4.jpeg')
gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# perform edge detection, find contours in the edge map, and sort the
# resulting contours from left-to-right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]
# initialize the list of contour bounding boxes and associated
# characters that we'll be OCR'ing
chars = []

In [ ]:
for c in cnts:
	(x, y, w, h) = cv2.boundingRect(c)
	if (w >= 5 and w <= 150) and (h >= 15 and h <= 120):
		roi = gray[y:y + h, x:x + w]
		thresh = cv2.threshold(roi, 0, 255,
			cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
		(tH, tW) = thresh.shape
		if tW > tH:
			thresh = imutils.resize(thresh, width=32)
		else:
			thresh = imutils.resize(thresh, height=32)
		(tH, tW) = thresh.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)
		# pad the image and force 32x32 dimensions
		padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
			left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
			value=(0, 0, 0))
		padded = cv2.resize(padded, (32, 32))
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)
		# update our list of characters that will be OCR'd
		chars.append((padded, (x, y, w, h)))

In [ ]:
# extract the bounding box locations and padded characters
boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")
# OCR the characters using our handwriting recognition model
preds = model.predict(chars)
# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [ ]:
c=0
count=0
p=0
for (pred, (x, y, w, h)) in zip(preds, boxes)	
	# find the index of the label with the largest corresponding
	# probability, then extract the probability and label
  i = np.argmax(pred)
  prob = pred[i]
  label = labelNames[i]
	# draw the prediction on the image
  # print("[INFO] {} - {:.2f}%".format(label, prob * 100))
  x1=0
  y1=0
  w1=0
  h1=0
	# print(label)
  for zi in range(0,25):
	  if(save[zi]==label):
	    # print(zi)
	    (x1,y1,w1,h1)=(bosx[zi])

	# cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 255, 0), 2)
	# cv2.putText(image1, label, (x - 10, y - 10),
	# 	cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
  img_res = image1[y:y+h,x:x+w]
  img_res = cv2.cvtColor(img_res, cv2.COLOR_BGR2GRAY)
  img_res = cv2.resize(img_res,(14,14))
  img_res1 = image[y1:y1+h1,x1:x1+w1]
	# print(img_res1)
  def mse(imageA, imageB):
	  err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	  err /= float(imageA.shape[0] * imageA.shape[1])
	  return err
	  
  def compare_images(imageA, imageB):
    # compute the mean squared error and structural similarity
    # index for the images
	  m = mse(imageA, imageB)
   	# ss = measure.compare_ssim(imageA, imageB)
	  s = measure.compare_ssim(imageA, imageB)
	  return s
  if(img_res1.size != 0):
	  
	  img_res1 = cv2.cvtColor(img_res1, cv2.COLOR_BGR2GRAY)
	  img_res1 = cv2.resize(img_res1,(14,14))
	  cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 255, 0), 2)
	  cv2.putText(image1, label, (x - 10, y - 10),
	  cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
	  cv2_imshow(image1)
	  ss=compare_images(img_res,img_res1)
	  c=c+ss
	  count=count+1
	  p=c/count

# print("pPPP")
print(p)
for (pred, (x, y, w, h)) in zip(preds, boxes):	
	# find the index of the label with the largest corresponding
	# probability, then extract the probability and label
  i = np.argmax(pred)
  prob = pred[i]
  label = labelNames[i]
	# draw the prediction on the image
  print("[INFO] {} - {:.2f}%".format(label, prob * 100))
  x1=0
  y1=0
  w1=0
  h1=0
	# print(label)
  for zi in range(0,25):
	  if(save[zi]==label):
	    print(zi)
	    (x1,y1,w1,h1)=(bosx[zi])
  img_res = image1[y:y+h,x:x+w]
  img_res = cv2.cvtColor(img_res, cv2.COLOR_BGR2GRAY)

  img_res = cv2.resize(img_res,(14,14))
  img_res1 = image[y1:y1+h1,x1:x1+w1]
	# print(img_res1)
  def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
	  err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	  err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
	  return err
	  
  def compare_images(imageA, imageB):
	  m = mse(imageA, imageB)
	  s = measure.compare_ssim(imageA, imageB)
	  print(s)
	  return s

  if(img_res1.size != 0):
	  
	  img_res1 = cv2.cvtColor(img_res1, cv2.COLOR_BGR2GRAY)
	  img_res1 = cv2.resize(img_res1,(14,14))
	  
	  # ss=compare_images(img_res,img_res1)
	  cv2_imshow(img_res)
	  cv2_imshow(img_res1)
	  ss=compare_images(img_res,img_res1)
	  if(ss>p):
	    print("Good")
	  else:
	    print("Bad")


    
	  

	  

	